In [1]:
import gammagl.transforms as T
import tensorlayerx as tlx
from gammagl.datasets import Planetoid
from gammagl.layers.conv import GCNConv

Using PyTorch backend.


Gammagl Global Config Info:
    TLX_BACKEND: torch
    DATASET_ROOT: C:\Users\mi\.ggl\datasets


In [2]:
dataset = Planetoid(root="./data", name="Cora", transform=T.NormalizeFeatures())

In [3]:
class GCN(tlx.nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats):
        super().__init__()
        self.conv1 = GCNConv(in_feats, hid_feats)
        self.conv2 = GCNConv(hid_feats, out_feats)
        self.relu = tlx.ReLU()

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = self.relu(x)
        x = self.conv2(x, edge_index)
        return x

In [4]:
model = GCN(dataset.num_features, 16, dataset.num_classes)

[TLX] Linear  linear_1: 16 No Activation
[TLX] Linear  linear_2: 7 No Activation


In [5]:
print(type(dataset))
print(type(dataset[0]))

<class 'gammagl.datasets.planetoid.Planetoid'>
<class 'gammagl.data.graph.Graph'>


In [8]:
data = dataset[0]
data

Graph(edge_index=[2, 10556], x=[2708, 1433], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [9]:
import torch
import torch.nn.functional as F

data = dataset[0]
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(200):
    pred = model(data.x, data.edge_index)
    loss = F.cross_entropy(pred[data.train_mask], data.y[data.train_mask])

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(loss.detach().item())

0.016058268025517464
0.0073593053966760635
0.00366101018153131
0.00209645158611238
0.0013734978856518865
0.000995747046545148
0.0007729133358225226
0.0006273755570873618
0.0005246467189863324
0.00044786621583625674
0.0003881828743033111
0.0003404758754186332
0.0003015416150446981
0.00026923572295345366
0.00024207636306528002
0.00021900089632254094
0.00019921416242141277
0.00018208214896731079
0.00016713661898393184
0.00015404692385345697


In [10]:
data

Graph(edge_index=[2, 10556], x=[2708, 1433], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [11]:
gcn = GCNConv(1433, 5)

[TLX] Linear  linear_3: 5 No Activation


In [13]:
gcn(data.x, data.edge_index)

E:\Anacodna3\envs\GML\lib\site-packages\tensorlayerx\backend\ops\torch_backend.py:1631: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  segment_ids = torch.tensor(segment_ids, dtype=torch.int64)


tensor([[ 1.1427e-02,  2.9385e-03,  3.1481e-03, -1.0216e-03,  8.4412e-03],
        [-3.3739e-03, -5.7997e-03,  7.3305e-04,  3.1416e-03,  2.2286e-02],
        [-6.7120e-04, -4.8504e-03, -6.0399e-03, -3.4293e-03,  3.0480e-03],
        ...,
        [ 4.3892e-02, -1.1945e-03, -6.2690e-03, -8.9860e-03,  4.6436e-03],
        [-7.0577e-03, -1.1329e-02,  2.6357e-03, -8.7571e-03, -1.1319e-05],
        [ 2.1782e-03, -1.1109e-02,  5.2946e-03, -6.9725e-03,  7.2902e-03]],
       grad_fn=<AddBackward0>)

In [18]:
lin = tlx.layers.Linear(1433, 5, W_init="xavier_uniform", b_init=None)

[TLX] Linear  linear_5: 1433 int


In [19]:
lin(data.x)

TypeError: 'int' object is not callable